In [4]:
import pandas as pd
import os

# Loading the data

In [5]:
from se_kge.default_predictor import predictor, embeddings_path, graph_path, model_path, mapping_path

In [6]:
print(f"""Loaded default predictor using paths:

embeddings: {embeddings_path}
graph: {graph_path}
model: {model_path}
mapping: {mapping_path}

""")

Loaded default predictor using paths:

embeddings: C:\Users\Rana Aldisi\Documents\GitHub\SE_KGE\resources\predictive_model\070819_node2vec_embeddings_complete01.embeddings
graph: C:\Users\Rana Aldisi\Documents\GitHub\SE_KGE\resources\chemsim_50_graphs\fullgraph_with_chemsim_50.edgelist
model: C:\Users\Rana Aldisi\Documents\GitHub\SE_KGE\resources\predictive_model\070819_node2vec_model_complete01.pkl
mapping: C:\Users\Rana Aldisi\Documents\GitHub\SE_KGE\resources\mapping\fullgraph_nodes_mapping.tsv




# Examples of different kinds of predictions with literature evidence

## side effect - target association

In [8]:
predictor.find_new_relation(
'8499', #EGFR
'6791', #Papulopustular rash
)
#PMID: 18165622

0.02084655412288939

In [13]:
predictor.find_new_relation(
'9438', #Histamine receptor H1
'331', #Drowsiness
)
#PMID: 26626077

0.022773732615890885

## drug- side effect association

In [9]:
predictor.find_new_relation(
'3534', #diazepam
'670', #Libido decreased
)
#PMID: 29888057

0.9969995536047346

In [11]:
predictor.find_new_relation(
'10270', #Cytarabine 
'1149', #Anaemia megaloblastic
)
# PMID: 23157436

0.0007503847088747362

## drug-target association

In [12]:
predictor.find_new_relation(
'12765', #Sertindole 
'9372',  #CHRM1 receptor
)
# PMID: 29942259 

0.014544440338331974

# Example of predicting relations using node2vec model and embeddings

In [ ]:
results = predictor.find_new_relations(
    entity_identifier='P00533',
    entity_type='phenotype',
    k=10,
)
print(results['query'])
results_df = pd.DataFrame(results['predictions'])
results_df